In [111]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [112]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('playground-series-s5e6')

print('Data source import complete.')


Data source import complete.


In [115]:
#import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder #for model selection strategies (like cross validation and data preprocessing)
import lightgbm as lgb # for building high-performance gradient boosting models, optimized for large datasets, low memory usage, and faster training

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier

In [121]:
train = pd.read_csv(f'{playground_series_s5e6_path}/train.csv')
test = pd.read_csv(f'{playground_series_s5e6_path}/test.csv')

print(train.shape)
print(test.shape)

(750000, 10)
(250000, 9)


In [125]:
print(X_train.dtypes)
print(y_train.dtype)


Temparature              int64
Humidity                 int64
Moisture                 int64
Nitrogen                 int64
Potassium                int64
Phosphorous              int64
Soil Type_Black           bool
Soil Type_Clayey          bool
Soil Type_Loamy           bool
Soil Type_Red             bool
Soil Type_Sandy           bool
Crop Type_Barley          bool
Crop Type_Cotton          bool
Crop Type_Ground Nuts     bool
Crop Type_Maize           bool
Crop Type_Millets         bool
Crop Type_Oil seeds       bool
Crop Type_Paddy           bool
Crop Type_Pulses          bool
Crop Type_Sugarcane       bool
Crop Type_Tobacco         bool
Crop Type_Wheat           bool
dtype: object
int64


In [126]:
#test cols
print(train.columns.tolist())

['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name']


In [127]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['target_enc'] = le.fit_transform(train['Fertilizer Name'])


In [128]:
print(le.classes_)


['10-26-26' '14-35-14' '17-17-17' '20-20' '28-28' 'DAP' 'Urea']


In [129]:
features = [c for c in train.columns if c not in ['id', 'Fertilizer Name', 'target_enc']]
X = train[features]
y = train['target_enc']


In [130]:
# 4. Set up cross-validation & LightGBM
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_probs = np.zeros((len(train), le.classes_.size))
test_probs = np.zeros((len(test), le.classes_.size))

# One-hot encode categorical features
X = pd.get_dummies(train[features], columns=['Soil Type', 'Crop Type'])
test_encoded = pd.get_dummies(test[features], columns=['Soil Type', 'Crop Type'])

# Align columns between training and test sets after one-hot encoding
train_cols = X.columns
test_cols = test_encoded.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    test_encoded[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X[c] = 0

test_encoded = test_encoded[train_cols]


for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    train_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params={'objective':'multiclass', 'num_class':le.classes_.size,
                'metric':'multi_logloss', 'learning_rate':0.1, 'verbosity':-1},
        train_set=train_data, valid_sets=[val_data],
        num_boost_round=200, callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=50)]
    )

    oof_probs[val_idx] = model.predict(X_val)
    test_probs += model.predict(test_encoded) / kf.n_splits

Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92258
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.9224
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92287
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92162
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[199]	valid_0's multi_logloss: 1.92281


In [131]:
pd.DataFrame(test_preds).head()


,0,1,2,3,4,5,6
0,0.138480,0.136932,0.127138,0.142565,0.161290,0.163096,0.130498
1,0.154011,0.128386,0.206362,0.164717,0.137909,0.093276,0.115339
2,0.151654,0.153297,0.113742,0.174236,0.145907,0.120991,0.140173
3,0.136151,0.220804,0.153262,0.112144,0.106316,0.130643,0.140679
4,0.158901,0.114763,0.140458,0.212115,0.144307,0.107677,0.121779


In [132]:
print(top3(test_preds[0]))


['DAP' '28-28' '20-20']


In [135]:
submission = pd.DataFrame([
    [i, " ".join(top3(row))] for i, row in zip(test_ids, test_preds)
], columns=["id", "Fertilizer Name"])
submission.to_csv("submission.csv", index=False)


In [137]:
submission.head()

,id,Fertilizer Name
0,750000,DAP 28-28 20-20
1,750001,17-17-17 20-20 10-26-26
2,750002,20-20 14-35-14 10-26-26
3,750003,14-35-14 17-17-17 Urea
4,750004,20-20 10-26-26 28-28
